import libraries

In [ ]:
import os
import json
import numpy as np
import pandas as pd
from utils import load_json

Helper Functions

In [4]:
def load_json(path):

    with open(path, "r") as f:

        return json.load(f)

Consolidate results

In [33]:
# Manually add in (hack)
folder = "./models_cached/morgan4_4096"
mist_folder = "../mist/models_cached/morgan4_4096"

all_folders = []
all_test_performances = {"loss": {}, "jaccard": {}} 

for model in os.listdir(folder):
    subfolder = os.path.join(folder, model)
    for checkpoint in os.listdir(subfolder):
        all_folders.append(os.path.join(subfolder, checkpoint))

for model in os.listdir(mist_folder):
    subfolder = os.path.join(mist_folder, model)
    for checkpoint in os.listdir(subfolder):
        all_folders.append(os.path.join(subfolder, checkpoint))

for f in all_folders:

    if "mist" in f: 
        idx = 5
    else:
        idx = 2 
    expt_name = f.replace(folder, "").split("/")[idx]
    test_performance = load_json(os.path.join(f, "test_performance.json"))
    for k, v in test_performance.items():
        all_test_performances[k][expt_name] = round(v, 5)

Format the table

In [34]:
results = pd.DataFrame.from_dict(all_test_performances)
results = results.reset_index()
results = results.rename(columns = {"index": "expt_name"})
results.loc[:, "dataset"] = results.loc[:, "expt_name"].apply(lambda x: x.split("_")[0]) 
results.loc[:, "model"] = results.loc[:, "expt_name"].apply(lambda x: x.split("_")[1]) 
results.loc[:, "split"] = results.loc[:, "expt_name"].apply(lambda x: "_".join(x.split("_")[3:])) 

results = results.loc[:, ["dataset", "model", "split", "loss", "jaccard"]]

results = results.sort_values(by = 'model', ascending = False)
results = results.sort_values(by = 'split', ascending = False)
results = results.sort_values(by = 'dataset', ascending = False)

results

,dataset,model,split,loss,jaccard
9,nist2023,formula,scaffold_vanilla,0.06786,0.18029
11,MSG,binned,scaffold_vanilla,0.09568,0.18942
13,MSG,binned,random,0.02889,0.61826
17,MSG,formula,inchikey_vanilla,0.08887,0.18680
16,MSG,binned,inchikey_vanilla,0.08890,0.19603
10,MSG,formula,random,0.03497,0.57630
19,MSG,mist,random,0.02474,0.53997
18,MSG,MS,inchikey_vanilla,0.08850,0.20217
15,MSG,MS,random,0.03245,0.58493
14,MSG,MS,scaffold_vanilla,0.08936,0.18995
